# Test conversions

- Quaternion <-> Euler angles
- Matrix <-> Euler Angles
- Quaternion <-> Matrix

In [14]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [2]:
import sys
sys.path.append("../..") 
from cryoem.conversions import *
from scipy.spatial.transform import Rotation as R

In [3]:
# Non-tf code for double-check
r = R.from_euler('zyz', [0.0, 0.0, 0.0])
r.as_quat()

array([0., 0., 0., 1.])

In [4]:
# Euler to Quaternions conversion
assert np.all(np.isclose(euler2quaternion([[0.0, 0.0, 0.0]]).numpy(), [0.0, 0.0, 0.0, 1.0], atol=1e-5)), euler2quaternion([[0.0, 0.0, 0.0]]).numpy()
assert np.all(np.isclose(euler2quaternion([[np.pi, 0.0, 0.0]]).numpy(), [0.0, 0.0, 1.0, 0.0], atol=1e-5)), euler2quaternion([[np.pi, 0.0, 0.0]]).numpy()
assert np.all(np.isclose(euler2quaternion([[0.0, np.pi, 0.0]]).numpy(), [0.0, 1.0, 0.0, 0.0], atol=1e-5)), euler2quaternion([[0.0, np.pi, 0.0]]).numpy()
assert np.all(np.isclose(euler2quaternion([[0.0, 0.0, np.pi]]).numpy(), [0.0, 0.0, 1.0, 0.0], atol=1e-5)), euler2quaternion([[0.0, 0.0, np.pi]]).numpy()
assert np.all(np.isclose(euler2quaternion([[np.pi, 0.0, np.pi]]).numpy(), [0.0, 0.0, 0.0, 1.0], atol=1e-5)), euler2quaternion([[np.pi, 0.0, np.pi]]).numpy()
assert np.all(np.isclose(euler2quaternion([[np.pi, np.pi, np.pi]]).numpy(), [0.0, 1.0, 0.0, 0.0], atol=1e-5)), euler2quaternion([[np.pi, np.pi, np.pi]]).numpy()

In [5]:
r = R.from_quat([0, 0, 0, 1])
r.as_euler('zyz', degrees=True)

/home/jelena/anaconda3/envs/protein_reconstruction/lib/python3.6/site-packages/scipy/spatial/transform/rotation.py:134: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  warnings.warn("Gimbal lock detected. Setting third angle to zero since"


array([0., 0., 0.])

In [6]:
# print(angles_true[0])
# e = angles_true[0]
# q = euler2quaternion([e])
# e_new = quaternion2euler([q])
# e_new.numpy()

In [7]:
# d_q(q, euler2quaternion(e_new)).numpy()

In [8]:
assert np.all(np.isclose(quaternion2euler([[0.0, 0.0, 0.0, 1.0]]).numpy(), [0.0, 0.0, 0.0], atol=1e-5)), quaternion2euler([[0.0, 0.0, 0.0, 1.0]]).numpy()


q = [0.0, 0.0, 1.0, 0.0]
a1 = [np.pi, 0.0, 0.0]
a2 = [0.0, 0.0, np.pi]
assert np.all(np.isclose(quaternion2euler([q]).numpy(), a1, atol=1e-5)) or np.all(np.isclose(quaternion2euler([q]).numpy(), a2, atol=1e-5)), quaternion2euler([q]).numpy()

In [9]:
assert np.all(np.isclose(quaternion2euler([[0.0, 1.0, 0.0, 0.0]]).numpy(), [0.0, np.pi, 0.0], atol=1e-5)), quaternion2euler([[0.0, 1.0, 0.0, 0.0]]).numpy()
# assert np.all(np.isclose(quaternion2euler([[0.0, 0.0, 1.0, 0.0]]).numpy(), [0.0, 0.0, np.pi], atol=1e-5)), quaternion2euler([[0.0, 0.0, 1.0, 0.0]]).numpy()

q = [0.0, 0.0, 0.0, -1.0]
a1 = [np.pi, 0.0, np.pi]
a2 = [0.0, 0.0, 0.0]
assert np.all(np.isclose(quaternion2euler([q]).numpy(),a1, atol=1e-5)) or np.all(np.isclose(quaternion2euler([q]).numpy(), a2, atol=1e-5)), quaternion2euler([q]).numpy()

q = [0.0, 1.0, 0.0, 0.0]
a1 = [np.pi, np.pi, np.pi]
a2 = [0.0, np.pi, 0.0]
assert np.all(np.isclose(quaternion2euler([q]).numpy(), a1, atol=1e-5)) or np.all(np.isclose(quaternion2euler([q]).numpy(), a2, atol=1e-5)), quaternion2euler([q]).numpy()

In [10]:
e = [0.0, np.pi, 0.0]
q = euler2quaternion([e])
e_new = quaternion2euler([q])
e_new.numpy()

array([[[-0.       ,  3.1415927,  0.       ]]], dtype=float32)

In [11]:
d_q(q, q).numpy()

array([7.88495335e-08])

In [12]:
def d_quat(quaternion1, quaternion2):
    dot_product = tf.reduce_sum(quaternion1 * quaternion2, axis=-1)
    # Ensure dot product is in range [-1. 1].
    eps = 8 * np.finfo(dot_product.dtype.as_numpy_dtype).eps
    dot_product *= (1.0 - eps)
    return 2.0 * tf.acos(tf.abs(dot_product))

In [13]:
d_quat(q, q).numpy()

array([0.00276214], dtype=float32)